In [1]:
import pandas as pd
import spacy
from itertools import zip_longest
import re

file_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\presidio_ddm\deny_list_dataset\deny_list_tokens_ddm.xlsx"
new_file_path = r"C:\Users\spolo\OneDrive\Documents\DOC\Work\TCS\Code\dynamic_data_masking\presidio_ddm\deny_list_dataset\deny_list_tokens_c4_v1.xlsx"

df = pd.read_excel(file_path, sheet_name="RACE")

nlp = spacy.load("fr_core_news_lg")
french_stopwords = nlp.Defaults.stop_words

In [2]:
import pandas as pd
from itertools import zip_longest

def augmentation(df, french_stopwords):
    new_df_columns = {}

    for col in df.columns:
        new_column = list(df[col].dropna())  # Remove NaN values

        # Split words by space
        new_column_split = [conv.split() for conv in new_column]
        new_column_split_joined = [item for lst in new_column_split for item in lst]
        new_column.extend(new_column_split_joined)

        # Remove French stopwords
        new_column = [word for word in new_column if word.lower() not in french_stopwords]

        # Handle hyphenated words
        split = [sub.split("-") for sub in new_column if '-' in sub]
        new_subcat = [item for lst in split for item in lst]
        new_column.extend(new_subcat)

        # Lowercase transformation
        new_column_lower = [word.lower() for word in new_column]
        new_column.extend(new_column_lower)

        # Remove stopwords again after modifications
        new_column = [word for word in new_column if word.lower() not in french_stopwords]

        # Convert to unique sorted list
        new_column = sorted(set(new_column))

        # Store results in dictionary (column_name: column_values)
        new_df_columns[col] = new_column

    # Make all columns the same length using zip_longest
    max_length = max(len(v) for v in new_df_columns.values())
    for col in new_df_columns:
        new_df_columns[col] += [None] * (max_length - len(new_df_columns[col]))  # Padding with None

    # Convert dictionary to DataFrame
    new_df = pd.DataFrame(new_df_columns)

    return new_df


new_df = augmentation(df, french_stopwords)

with pd.ExcelWriter(new_file_path, mode='a', if_sheet_exists='new', engine='openpyxl') as writer:
    new_df.to_excel(writer, sheet_name='RACE', index=False)

In [17]:
def clean_races_category(subcategory):
    subcat = list(racial[subcategory].dropna())
    subcat_lower = [sub.lower() for sub in subcat]
    subcat.extend(subcat_lower)
    split_subcat = [sub.split("-") for sub in subcat if '-' in sub]
    new_subcat = [item for lst in split_subcat for item in lst]
    subcat.extend(new_subcat)
    subcat = [re.sub(r'\s*\(.*?\)', '', item) for item in subcat]
    subcat = set(subcat)
    print(len(subcat))
    return subcat


# RACES

In [18]:
races = clean_races_category('Race')

543


# NATIONALITY

In [19]:
nationalities = clean_races_category('Nationality')

1295


# REGIONAL

In [20]:
regional = clean_races_category('Regional')

716


# LANGUAGE

In [21]:
languages = clean_races_category('Language & Dialect-Based Identity')

107


# CULTURE

In [22]:
culture = clean_races_category('Cultural & Heritage-Based Identity')

208
